# Basic RAG (Retrieval Augmented Generation)

In [ ]:
# ! pip install faiss-cpu "mistralai>=0.1.2"

### API key 불러오기
이전 파일들을 참고해 주세요!

In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

### 데이터 불러오기

- https://www.deeplearning.ai/the-batch/ 이 사이트를 방문하셔도 좋습니다.
- 혹은 어떤 기사의 URL을 가져오셔도 좋고요.

### BeautifulSoup을 이용하여 기사 parsing하기
여기는 그냥 텍스트를 가져온다 생각하고 넘어가겠습니다.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get(
    "https://www.deeplearning.ai/the-batch/a-roadmap-explores-how-ai-can-detect-and-mitigate-greenhouse-gases/"
)
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
print(text)

### Optionally, 위에서 획득한 텍스트를 파일로 저장해도 됩니다
- 다음 강의에서는 텍스트 파일을 챗 인터페이스에 업로드하는 방법을 다룹니다.

In [ ]:
file_name = "AI_greenhouse_gas.txt"
with open(file_name, 'w') as file:
    file.write(text)

### Chunking
텍스트를 일정한 단위로 잘라내는 것을 뜻합니다.

여기서는 512 글자 단위로 텍스트 덩어리를 구성합니다.

In [ ]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [ ]:
len(chunks)

### Get embeddings of the chunks
각 덩어리의 임베딩 벡터를 획득합니다.

이를 위해서 mistral의 embedding 모델을 사용합니다.

즉, embedding 모델에 512글자 단위로 구분된 chunk를 입력으로 제공하면 일정한 길이(1024)의 벡터가 반환됩니다.

In [ ]:
import os
from mistralai.client import MistralClient


def get_text_embedding(txt):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=txt)
    return embeddings_batch_response.data[0].embedding

In [ ]:
import numpy as np

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [ ]:
text_embeddings

In [ ]:
len(text_embeddings[0])

### 벡터 데이터베이스에 저장하기
- 본 강의에서는 [Faiss](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)라는 라이브러리를 사용합니다.

In [ ]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

### 유저 쿼리 임베딩하기
위에서와 동일한 embedding 모델을 사용하여 유저의 쿼리(텍스트)도 벡터로 변환합니다.

In [ ]:
question = "What are the ways that AI can reduce emissions in Agriculture?"
question_embeddings = np.array([get_text_embedding(question)])

In [ ]:
question_embeddings

### 쿼리와 가장 비슷한 chunk 탐색하기
기존에 embedding한 vector를 기준으로 유저의 query를 embedding한 vector와 비교하여 가장 유사한 것을 찾아냅니다.

여기에서는 최대 2개의 chunk를 반환하도록 설정되어 있습니다.

In [ ]:
D, I = index.search(question_embeddings, k=2)
print(I)

In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

검색을 통해 획득한 chunk를 `retrieved_chunk`라는 변수에 context로 넣어주게 됩니다.

In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

이전에 사용한 것과 동일한 mistral 함수입니다.

In [ ]:
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [ ]:
response = mistral(prompt)
print(response)

## RAG + Function calling
이와 같은 RAG의 구조를 직전의 강의에서 배운 Function calling과 결합합니다.

RAG도 특정 조건을 충족하는 경우에만 활용 가능한, 일종의 'function'처럼 인식시킬 수 있습니다.

따라서 텍스트를 일정한 길이의 chunk로 쪼갠 뒤 벡터 데이터베이스를 만들고, 유저 쿼리를 기반으로 유사도가 높은 chunk를 반환한 뒤 적절한 답변을 생성하는 일련의 과정을 하나의 function으로 만들어 줍니다.

In [ ]:
def qa_with_context(text, question, chunk_size=512):
    # split document into chunks
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    # load into a vector database
    text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieve relevant text chunks

    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    response = mistral(prompt)
    return response

In [ ]:
I.tolist()

In [ ]:
I.tolist()[0]

본 예시에서는 하나의 function을 아래에서 쓰고 있지만, 활용하고자 하는 function의 종류가 많을수록 유용한 기능입니다.

In [ ]:
import functools

names_to_functions = {"qa_with_context": functools.partial(qa_with_context, text=text)}

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "qa_with_context",
            "description": "Answer user question by retrieving relevant context", # 수정 가능
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "user question",
                    }
                },
                "required": ["question"],
            },
        },
    },
]

In [ ]:
question = """
What are the ways AI can mitigate climate change in transportation?
"""

client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)

response = client.chat(
    model="mistral-large-latest",
    messages=[ChatMessage(role="user", content=question)],
    tools=tools,
    tool_choice="any", # auto, none
)

response

tools에 정의한 'description'과 response의 'tool_choice'에 따라 결과가 천차만별일 수 있습니다.

우선 'tool_choice'가 any로 설정된 경우 유저의 query와 상관 없이 반드시 함수를 호출하게 됩니다.

위 예시에서 주어진 question은 우리가 위에서 정의한 RAG를 활용할 필요가 전혀 없음에도 불구하고 이를 억지로 호출하여 엉뚱한 답변이 생성되는 것이죠.

이를 방지하기 위해 tool_choice를 'auto'로 설정하는 것이 권장됩니다.

또한 언어모델이 description을 바탕으로 보다 구체적이고 명확한 상황에서 tool 사용을 결정할 수 있도록 하는 것이 좋습니다.

In [ ]:
tool_function = response.choices[0].message.tool_calls[0].function
tool_function

In [ ]:
tool_function.name

In [ ]:
import json

args = json.loads(tool_function.arguments)
args

In [ ]:
function_result = names_to_functions[tool_function.name](**args)
function_result

## More about RAG
chunking과 retrieval 방법에 대한 디테일한 내용을 공부해보고 싶다면 아래 강의들을 참고해 보세요:
- [Advanced Retrieval for AI with Chroma](https://learn.deeplearning.ai/courses/advanced-retrieval-for-ai/lesson/1/introduction)
  - Sentence window retrieval
  - Auto-merge retrieval
- [Building and Evaluating Advanced RAG Applications](https://learn.deeplearning.ai/courses/building-evaluating-advanced-rag)
  - Query Expansion
  - Cross-encoder reranking
  - Training and utilizing Embedding Adapters
